In [5]:
print('Hello World!')

Hello World!


In [6]:
# Changing this variable changes which file to parse.
# Makes testing/presenting easier, since less code has to be changed.
FILENUM = 1

In [9]:
from random import shuffle, random

from src.ParentSelectionMethods import tournament as PSM
from src.RecombinationMethods import recombination_cut_crossover as RM
from src.MutationMethods import permutation_swap as MM
from src.SurvivorSelectionMethods import replacement as SSM
from src.Setups.TSP.TSP import read_tsp_file as parse_file
from src.Setups.TSP.TSP import random_initialization as initialize
from src.Setups.TSP.TSP import euclidean_distance as eval_fitness

genome_length       =      parse_file(FILENUM)
generation_limit    =      100

population_size     =      60
mating_pool_size    =      population_size//2 if (population_size//2) % 2 == 0 else (population_size//2)+1  # has to be even
tournament_size     =      population_size//10

mutation_rate       =      0.2
crossover_rate      =      0.9
crossover_point     =      genome_length//3



In [ ]:
def evolution_algorithm(maximize, print_gens=True, display_freq=None):
    if display_freq is None: display_freq = generation_limit
    
    # Modular function declarations
    op = max if maximize else min

    # Initialize Population
    population = initialize(population_size, genome_length)
    fitness = [eval_fitness(i) for i in population]

    # Evolution starts
    for generation in range(generation_limit):

        # Generation Info
        if print_gens:
            print("Generation: {}\n  Best fitness: {}\n  Avg. fitness: {}".format(
                generation+1, op(fitness), sum(fitness)/len(fitness))
            )

        # Parent Selection
        parents_index = PSM(fitness, mating_pool_size, tournament_size, op)
        shuffle(parents_index)

        # Recombination
        offspring = []
        for i in range(0, mating_pool_size, 2):
            if random() < crossover_rate:
                off1, off2 = RM(population[parents_index[i]], population[parents_index[i+1]], crossover_point)
            else:
                off1, off2 = population[parents_index[i]].copy(), population[parents_index[i+1]].copy()
            offspring.append(off1)
            offspring.append(off2)

        # Mutations Selection
        offspring = [MM(i) if random() < mutation_rate else i for i in offspring]
        offspring_fitness = [eval_fitness(i) for i in offspring]

        # Survivor Selection
        population, fitness = SSM(population, fitness, offspring, offspring_fitness, op)
        
        if generation % display_freq == 0:
            # TODO - Cool display things go here.
            pass
    # Evolution ends
        
    # Final Fitness Info
    op_fit = op(fitness)
    optimal_solutions = [i + 1 for i in range(population_size) if fitness[i] == op_fit]
    print("Best solution fitness:", op_fit, "\nNumber of optimal solutions: ", len(optimal_solutions), '/', population_size)
    print("Best solution indexes:", optimal_solutions)